<a href="https://colab.research.google.com/github/nastya-biran/scheduler/blob/main/%D0%A1%D0%BE%D1%81%D1%82%D0%B0%D0%B2%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5_%D1%80%D0%B0%D1%81%D0%BF%D0%B8%D1%81%D0%B0%D0%BD%D0%B8%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [112]:
!pip install pulp 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [113]:
from pulp import *
import numpy as np
import pandas as pd

In [114]:
teachers_data = pd.read_csv("Teachers.csv")
groups_data = pd.read_csv("Groups.csv")
limits_data = pd.read_csv("Teachers_limits.csv")

In [115]:
teachers_data.head(5)

,Преподаватель,Зельеварение,Защита от темных сил,Трансфигурация,Понедельник,Вторник,Среда,Четверг,Пятница,Суббота
0,Северус Снейп,1,1,0,NaN,"16.20-17.40, 18.10-19.30",NaN,"9.30-10.50, 11.10-12.30",NaN,NaN
1,Минерва Макгоналл,0,1,1,"11.10-12.30, 13.00-14.20, 14.40-16.00, 16.20-1...",NaN,NaN,NaN,NaN,NaN
2,Альбус Дамблдор,1,0,1,"11.10-12.30, 13.00-14.20",NaN,NaN,"9.30-10.50, 11.10-12.30",NaN,NaN


In [116]:
groups_data.head(5)

,Номер группы,Зельеварение,Защита от темных сил,Трансфигурация
0,1,1,1,0
1,2,1,1,0
2,3,0,1,1
3,4,1,1,1
4,5,1,0,1


In [117]:
limits_data.head(5)

,Преподаватель,Все пары в один день,Дни без пар
0,Северус Снейп,NaN,"Понедельник, Среда, Суббота"
1,Минерва Макгоналл,Да,NaN
2,Альбус Дамблдор,NaN,"Вторник, Пятница, Суббота"


In [118]:
all_slots = ['9.30-10.50', '11.10-12.30', '13.00-14.20', '14.40-16.00', '16.20-17.40', '18.10-19.30', '19.40-21.00']
all_days = ['Понедельник', 'Вторник', 'Среда', 'Четверг', 'Пятница', 'Суббота']
all_subjects = list(groups_data.columns.values)[1:]
all_teachers = list(pd.unique(teachers_data["Преподаватель"]))
all_groups = list(pd.unique(groups_data["Номер группы"]))
print(all_subjects)
print(all_teachers)
print(all_groups)

['Зельеварение', 'Защита от темных сил', 'Трансфигурация']
['Северус Снейп', 'Минерва Макгоналл', 'Альбус Дамблдор']
[1, 2, 3, 4, 5, 6]


In [119]:
def get_subjects_by_teacher(teacher):
  teacher_data = teachers_data[teachers_data["Преподаватель"] == teacher]
  teacher_data = teacher_data.filter(all_subjects)
  available_subjects = list(teacher_data.columns[teacher_data.isin([1]).any()].values)
  subject_inds = [all_subjects.index(subject) for subject in available_subjects]
  return subject_inds

def get_groups_by_subject(subject):
  subject_data = groups_data[groups_data[subject] == 1]
  available_groups = list(pd.unique(subject_data["Номер группы"]))
  group_inds = [all_groups.index(group) for group in available_groups]
  return group_inds

def get_answers(teacher):
  teacher_data = teachers_data[teachers_data["Преподаватель"] == teacher]
  answers = []
  for day in all_days:
    if pd.isna(teacher_data.iloc[0][day]):
      answers.append(None)
    else:
      slot_inds = [all_slots.index(slot) for slot in teacher_data.iloc[0][day].split(", ")]
      answers.append(slot_inds)
  return answers



def get_subjects_by_group(group):
  group_data = groups_data[groups_data["Номер группы"] == group]
  group_data = group_data.filter(all_subjects)
  needed_subjects = list(group_data.columns[group_data.isin([1]).any()].values)
  subject_inds = [all_subjects.index(subject) for subject in needed_subjects]
  return subject_inds


def get_teachers_by_subject(subject):
  subject_data = teachers_data[teachers_data[subject] == 1]
  available_teachers = list(pd.unique(subject_data["Преподаватель"]))
  teacher_inds = [all_teachers.index(teacher) for teacher in available_teachers]
  return teacher_inds



In [120]:
pref_table = np.zeros((len(groups_data), len(all_subjects), len(teachers_data), len(all_days), len(all_slots)))

for k, teacher in enumerate(all_teachers):
  available_subjects = get_subjects_by_teacher(teacher)
  available_slots = get_answers(teacher)
  for j in available_subjects:
    available_groups = get_groups_by_subject(all_subjects[j])
    for i in available_groups:
      for l, slots_for_day in enumerate(available_slots):
        if slots_for_day is not None:
          for m in slots_for_day:
            pref_table[i, j, k, l, m] = 1


In [133]:
prob = LpProblem("schedule", LpMaximize)

attends = LpVariable.dicts("x", ((i, j, k, l, m)
                                  for i in range(len(all_groups))
                                  for j in range(len(all_subjects))
                                  for k in range(len(all_teachers))
                                  for l in range(len(all_days))
                                  for m in range(len(all_slots))),
                            cat="Binary")

#Условие 2.1
for i in range(len(all_groups)):
  needed_subjects = get_subjects_by_group(all_groups[i])
  for j in needed_subjects:
    #Группа должна посещать по одной паре каждого нужного предмета у кого-то из учителей, которые ведут этот предмет
    available_teachers = get_teachers_by_subject(all_subjects[j])
    prob += lpSum([attends[(i, j, k, l, m)]
                  for k in available_teachers
                  for l in range(len(all_days))
                  for m in range(len(all_slots))]) == 1
    #Нельзя ходить на пару к учителю, который не ведет этот предмет
    unavailable_teachers = list(set(range(len(all_teachers))) - set(available_teachers))
    prob += lpSum([attends[(i, j, k, l, m)]
                  for k in unavailable_teachers
                  for l in range(len(all_days))
                  for m in range(len(all_slots))]) == 0
  #Нельзя ходить на ненужные предметы
  unneeded_subjects = list(set(range(len(all_subjects))) - set(needed_subjects))
  for j in unneeded_subjects:
    prob += lpSum([attends[(i, j, k, l, m)]
                  for k in range(len(all_teachers))
                  for l in range(len(all_days))
                  for m in range(len(all_slots))]) == 0


#Условие 2.2
#Группа посещает не более одного слота одновременно
for i in range(len(all_groups)):
  for l in range(len(all_days)):
    for m in range(len(all_slots)):
      prob += lpSum([attends[(i, j, k, l, m)]
                     for j in range(len(all_subjects))
                     for k in range(len(all_teachers))]) <= 1


#Условие 3.1
#Преподаватель ведет занятие не более чем у одной группы одновременно
for k in range(len(all_teachers)):
  for l in range(len(all_days)):
    for m in range(len(all_slots)):
      prob += lpSum([attends[(i, j, k, l, m)]
                     for i in range(len(all_groups))
                     for j in range(len(all_subjects))]) <= 1


#Условие 3.3
#Отсутствие пар в определенные дни
for k in range(len(all_teachers)):
  forbidden_days_answer = limits_data[limits_data["Преподаватель"] == all_teachers[k]].iloc[0]["Дни без пар"]
  if pd.notna(forbidden_days_answer):
    forbidden_days = [all_days.index(day) for day in forbidden_days_answer.split(", ")]
    for l in forbidden_days:
      prob += lpSum([attends[(i, j, k, l, m)]
                    for i in range(len(all_groups))
                    for j in range(len(all_subjects))
                    for m in range(len(all_slots))]) == 0

#Условие 3.4
#Все пары в один день
y = LpVariable.dicts("y", ((k, l)
                    for k in range(len(all_teachers))
                    for l in range(len(all_days))),
               cat="Binary")
M = 1e5
for k in range(len(all_teachers)):
  for l in range(len(all_days)):
    prob += M * y[(k, l)] >= lpSum([attends[(i, j, k, l, m)]
                                    for i in range(len(all_groups))
                                    for j in range(len(all_subjects))
                                    for m in range(len(all_slots))])
    

for k in range(len(all_teachers)):
  in_one_day_answer = limits_data[limits_data["Преподаватель"] == all_teachers[k]]
  if in_one_day_answer.iloc[0]["Все пары в один день"] == "Да": 
    prob += lpSum([y[(k, l)] for l in range(len(all_days))]) == 1



#Целевая функция -- максимизация довольных преподавателей
tgt1 = LpVariable("tgt1", 
                  cat="Integer")

prob += lpSum([pref_table[i, j, k, l, m] * attends[(i, j, k, l, m)]
               for i in range(len(all_groups))
               for j in range(len(all_subjects))
               for k in range(len(all_teachers)) 
               for l in range(len(all_days)) 
               for m in range(len(all_slots))]) == tgt1
prob += tgt1

In [134]:
prob.solve()

1

In [135]:
print("Значение целевой функции:", prob.objective.value())

Значение целевой функции: 12.0


In [137]:
prob_res = prob.variablesDict()
res = np.chararray((len(all_teachers), len(all_slots), len(all_days)), itemsize=100, unicode=True).astype(str)
res[:] = ""
for k in range(len(all_teachers)):
  print(f"\nПреподаватель:{all_teachers[k]}")
  for i in range(len(all_groups)):
    for j in range(len(all_subjects)):
      for l in range(len(all_days)):
        for m in range(len(all_slots)):
          if prob_res[f"x_({i},_{j},_{k},_{l},_{m})"].value() == 1:
            res[k, m, l] += str(all_groups[i]) + ',' + all_subjects[j]
            print(f"{all_days[l]} {all_slots[m]} Группа: {res[k, m, l]}")

pd.DataFrame(res[1], columns=all_days, index=all_slots)


Преподаватель:Северус Снейп
Вторник 18.10-19.30 Группа: 2,Зельеварение
Вторник 16.20-17.40 Группа: 2,Защита от темных сил
Четверг 9.30-10.50 Группа: 4,Зельеварение
Четверг 11.10-12.30 Группа: 4,Защита от темных сил

Преподаватель:Минерва Макгоналл
Понедельник 16.20-17.40 Группа: 1,Защита от темных сил
Понедельник 14.40-16.00 Группа: 3,Защита от темных сил
Понедельник 9.30-10.50 Группа: 3,Трансфигурация
Понедельник 13.00-14.20 Группа: 5,Трансфигурация
Понедельник 11.10-12.30 Группа: 6,Защита от темных сил

Преподаватель:Альбус Дамблдор
Понедельник 13.00-14.20 Группа: 1,Зельеварение
Понедельник 11.10-12.30 Группа: 4,Трансфигурация
Четверг 9.30-10.50 Группа: 5,Зельеварение
Четверг 11.10-12.30 Группа: 6,Трансфигурация


,Понедельник,Вторник,Среда,Четверг,Пятница,Суббота
9.30-10.50,"3,Трансфигурация",,,,,
11.10-12.30,"6,Защита от темных сил",,,,,
13.00-14.20,"5,Трансфигурация",,,,,
14.40-16.00,"3,Защита от темных сил",,,,,
16.20-17.40,"1,Защита от темных сил",,,,,
18.10-19.30,,,,,,
19.40-21.00,,,,,,
